In [1]:
import pandas as pd
import numpy as np
import os
from collections import defaultdict
import re

In [ ]:
data_path = '/data/baiyimeng/dataset/aliccp'
common_features_train_csv = os.path.join(data_path, "common_features_train.csv")
common_features_test_csv = os.path.join(data_path, "common_features_test.csv")
sample_skeleton_train_csv = os.path.join(data_path, "sample_skeleton_train.csv")
sample_skeleton_test_csv = os.path.join(data_path, "sample_skeleton_test.csv")

In [3]:
fea_cols = ['common_feature_index', 'feature_num2', 'feature_list2']
fea_train = pd.read_csv(common_features_train_csv, header=None, names=fea_cols, iterator=True,chunksize = 100000)
fea_test = pd.read_csv(common_features_test_csv, header=None, names=fea_cols, iterator=True,chunksize = 100000)

In [4]:
sample_cols = ['sample_id', 'click', 'conversion', 'common_feature_index', 'feature_num1', 'feature_list1']
sample_train = pd.read_csv(sample_skeleton_train_csv, header=None, names=sample_cols, iterator=True,chunksize = 2500000)
sample_test = pd.read_csv(sample_skeleton_test_csv, header=None, names=sample_cols, iterator=True,chunksize = 2500000)

In [5]:
user_features = ['101','121','122','124','125','126','127','128','129','150_14','127_14','109_14','110_14']
item_features = ['205','206','207', '210','216','508','509','702','853','301']

In [6]:
def feature_list_split(x):
    feature_dict = defaultdict(list)
    for fea in x.split('\x01'):
        field = re.split('\x02', fea)
        feature_dict[field[0]].append(field[1])
    return feature_dict

In [7]:
def user_fea_fn(data):
    data['fea_dict2'] =  data['feature_list2'].map(feature_list_split)
    data['101'] = data['fea_dict2'].map(lambda x: int(x['101'][0].split('\x03')[0]) if '101' in x else 0) # 用户ID
    data['121'] = data['fea_dict2'].map(lambda x: int(x['121'][0].split('\x03')[0]) if '121' in x else 0) # 用户的一种分类ID
    data['122'] = data['fea_dict2'].map(lambda x: int(x['122'][0].split('\x03')[0]) if '122' in x else 0) # 用户的一种分类ID
    data['124'] = data['fea_dict2'].map(lambda x: int(x['124'][0].split('\x03')[0]) if '124' in x else 0) # 用户性别分类ID
    data['125'] = data['fea_dict2'].map(lambda x: int(x['125'][0].split('\x03')[0]) if '125' in x else 0) # 用户年龄分类ID
    data['126'] = data['fea_dict2'].map(lambda x: int(x['126'][0].split('\x03')[0]) if '126' in x else 0) # 用户消费水平分类I    
    data['127'] = data['fea_dict2'].map(lambda x: int(x['127'][0].split('\x03')[0]) if '127' in x else 0) # 用户消费水平分类II
    data['128'] = data['fea_dict2'].map(lambda x: int(x['128'][0].split('\x03')[0]) if '128' in x else 0) # 用户是否就业
    data['129'] = data['fea_dict2'].map(lambda x: int(x['129'][0].split('\x03')[0]) if '129' in x else 0) # 用户地理信息分类ID
    data['150_14'] = data['fea_dict2'].map(lambda x: x['150_14']) # 用户意图ID以及用户在该意图上的历史行为累积数量*
    data['127_14'] = data['fea_dict2'].map(lambda x: x['127_14']) # 商品品牌ID以及用户在该店铺上的历史行为累积数量*
    data['109_14'] = data['fea_dict2'].map(lambda x: x['109_14']) # 商品类目ID以及用户在该类目上的历史行为累积数量*
    data['110_14'] = data['fea_dict2'].map(lambda x: x['110_14']) # 商品店铺ID以及用户在该店铺上的历史行为累积数量*
    data.drop(columns=['feature_num2','feature_list2', 'fea_dict2'], inplace=True)
    # data.drop(columns=['150_14', '127_14', '109_14', '110_14'], inplace=True)

In [8]:
def item_fea_fn(data):
    data['fea_dict1'] =  data['feature_list1'].map(feature_list_split)
    data['205'] = data['fea_dict1'].map(lambda x: int(x['205'][0].split('\x03')[0]) if '205' in x else 0) # 商品ID
    data['206'] = data['fea_dict1'].map(lambda x: int(x['206'][0].split('\x03')[0]) if '206' in x else 0) # 商品所属类目ID
    data['207'] = data['fea_dict1'].map(lambda x: int(x['207'][0].split('\x03')[0]) if '207' in x else 0) # 商品所属店铺ID
    data['210'] = data['fea_dict1'].map(lambda x:[i.split('\x03')[0] for i in x['210']]) # 商品关联用户意图ID：多值
    data['216'] = data['fea_dict1'].map(lambda x: int(x['216'][0].split('\x03')[0]) if '216' in x else 0) # 商品的品牌ID
    data['508'] = data['fea_dict1'].map(lambda x:x['508'][0].split('\x03')[1] if len(x['508'])>0 else np.NaN) # 商品类目ID以及用户在该类目上的历史行为累积数量*和206域商品所属类目ID的组合特征：浮点值 商品所属类目ID
    data['509'] = data['fea_dict1'].map(lambda x:x['509'][0].split('\x03')[1] if len(x['509'])>0 else np.NaN) # 110_14和207域的组合特征
    data['702'] = data['fea_dict1'].map(lambda x:x['702'][0].split('\x03')[1] if len(x['702'])>0 else np.NaN) # 127_14和216域的组合特征
    data['853'] = data['fea_dict1'].map(lambda x:x['853']) # 150_14和210域的组合特征：多值，浮点值
    data['301'] = data['fea_dict1'].map(lambda x: int(x['301'][0].split('\x03')[0]) if '301' in x else 0) # 业务场景信息的一种分类表示
    data.drop(columns=['feature_num1','feature_list1', 'fea_dict1'], inplace=True)
    # data.drop(columns=['210','508','509','702','853'], inplace=True)

In [9]:
for i, chunk_df in enumerate(fea_train):
    print(i,chunk_df.shape)
    user_fea_fn(chunk_df)
    chunk_df.to_pickle(os.path.join(data_path, f'fea_train{i}.pkl'))

0 (100000, 3)
1 (100000, 3)
2 (100000, 3)
3 (100000, 3)
4 (100000, 3)
5 (100000, 3)
6 (100000, 3)
7 (30600, 3)


In [10]:
for i, chunk_df in enumerate(fea_test):
    print(i,chunk_df.shape)
    user_fea_fn(chunk_df)
    chunk_df.to_pickle(os.path.join(data_path, f'fea_test{i}.pkl'))

0 (100000, 3)
1 (100000, 3)
2 (100000, 3)
3 (100000, 3)
4 (100000, 3)
5 (100000, 3)
6 (100000, 3)
7 (100000, 3)
8 (84212, 3)


In [11]:
for i, chunk_df in enumerate(sample_train):
    print(i, chunk_df.shape)
    item_fea_fn(chunk_df)
    chunk_df.to_pickle(os.path.join(data_path, f'sample_train{i}.pkl'))

0 (2500000, 6)
1 (2500000, 6)
2 (2500000, 6)
3 (2500000, 6)
4 (2500000, 6)
5 (2500000, 6)
6 (2500000, 6)
7 (2500000, 6)
8 (2500000, 6)
9 (2500000, 6)
10 (2500000, 6)
11 (2500000, 6)
12 (2500000, 6)
13 (2500000, 6)
14 (2500000, 6)
15 (2500000, 6)
16 (2300135, 6)


In [12]:
for i, chunk_df in enumerate(sample_test):
    print(i, chunk_df.shape)
    item_fea_fn(chunk_df)
    chunk_df.to_pickle(os.path.join(data_path, f'sample_test{i}.pkl'))

0 (2500000, 6)
1 (2500000, 6)
2 (2500000, 6)
3 (2500000, 6)
4 (2500000, 6)
5 (2500000, 6)
6 (2500000, 6)
7 (2500000, 6)
8 (2500000, 6)
9 (2500000, 6)
10 (2500000, 6)
11 (2500000, 6)
12 (2500000, 6)
13 (2500000, 6)
14 (2500000, 6)
15 (2500000, 6)
16 (2500000, 6)
17 (516840, 6)


In [13]:
fea_train = pd.concat([pd.read_pickle(os.path.join(data_path, f'fea_train{i}.pkl')) for i in range(8)])

In [14]:
fea_train

,common_feature_index,101,121,122,124,125,126,127,128,129,150_14,127_14,109_14,110_14
0,84dceed2e3a667f8,31319,3438687,3438762,3438769,3438774,3438779,3438782,3864885,3864888,"[38788823.28343, 39609392.44235, 39005791.1...","[37810410.69315, 38509350.69315, 38502350.6...","[4508772.30259, 4474141.79176, 4464420.6931...","[9599610.69315, 6262140.69315, 28695011.098..."
1,0000350f0c2121e7,392326,3438725,3438760,3438769,3438772,3438778,3438782,3864885,3864888,"[38945654.37156, 38979711.50408, 38805771.7...","[37162241.94591, 35146270.69315, 37728710.6...","[4475531.38629, 4459952.19722, 4502472.8332...","[10387361.60944, 22546570.69315, 4581380.69..."
2,000091a89d1867ab,0,3438658,3438761,3438769,3438773,0,3438781,3864885,3864889,[],[],[],[]
3,0001a4114b0ae8bf,65187,3438658,3438761,3438769,3438773,0,3438782,3864885,0,"[39166842.3979, 39407981.07056, 38923681.62...","[37603130.69315, 38502350.69315, 37729840.6...","[4491770.69315, 4508771.09861, 4503002.7080...","[28069190.69315, 19297282.3979, 10438401.09..."
4,0001def19d7cb335,241189,3438685,3438762,3438769,3438774,3438778,3438782,3864885,3864888,"[39091500.84715, 39330134.44265, 39340833.3...","[35307781.09861, 36899321.79176, 34975950.6...","[4490992.70805, 4556761.09861, 4493601.6094...","[6553780.69315, 10285720.69315, 11922140.69..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730595,fffc363b2021c5d4,358830,3438696,3438763,3438769,3438775,3438778,3438782,3864885,3864887,"[38767720.77334, 39321982.60269, 39044581.5...","[37528450.69315, 38581942.19722, 38322430.6...","[4519180.69315, 4519042.19722, 4511122.9444...","[33219960.69315, 30151110.69315, 6698041.09..."
730596,fffc4141b0cdf8f9,316218,3438658,3438757,3438768,3438774,0,3438780,3864885,0,"[38802201.84577, 39262820.07973, 38961720.1...","[35424610.69315, 38242410.69315, 35900141.0...","[4469661.09861, 4511170.69315, 4466221.3862...","[10938881.38629, 14347780.69315, 12148491.0..."
730597,fffd7a838fb0c2c7,90273,3438658,3438761,3438769,3438773,0,3438782,3864885,0,"[38891851.81921, 38685891.50408, 39159931.9...","[34397790.69315, 37684101.79176, 35520681.0...","[4514290.69315, 4509372.3979, 4524581.94591...","[12436070.69315, 27480461.79176, 9322241.38..."
730598,fffe19a4e660d1a8,55970,3438685,3438762,3438769,3438774,3438778,3438782,3864885,3864888,"[39425262.22462, 39016121.83258, 38722761.2...","[38254581.60944, 34670361.09861, 37971480.6...","[4514280.69315, 4501740.69315, 4508781.3862...","[32060203.04452, 19297380.69315, 21827880.6..."


In [23]:
sample_train = pd.concat([pd.read_pickle(os.path.join(data_path, f'sample_train{i}.pkl')) for i in range(17)])

In [25]:
user_features = ['101','121','122','124','125','126','127','128','129'] + ['common_feature_index']
item_features = ['205','206','207','216','301'] + ['common_feature_index', 'sample_id', 'click']

In [27]:
train_all = sample_train[item_features].merge(fea_train[user_features], how='left', on='common_feature_index')
train_all

,205,206,207,216,301,common_feature_index,sample_id,click,101,121,122,124,125,126,127,128,129
0,4186222,8316799,8416205,9154780,9351665,bacff91692951881,1,0,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887
1,4186855,8316676,8482726,9188757,9351665,bacff91692951881,2,0,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887
2,4479003,8316590,8542197,9165876,9351665,bacff91692951881,3,1,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887
3,4933915,8316177,8529759,9323908,9351665,bacff91692951881,4,0,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887
4,4945663,8316589,8754197,9172179,9351665,bacff91692951881,5,0,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42300130,7886537,8317499,9014187,0,9351666,c7155f38c0251cf4,42300131,0,423636,0,0,0,0,0,0,0,0
42300131,7918046,8315276,8532063,9318221,9351666,c7155f38c0251cf4,42300132,0,423636,0,0,0,0,0,0,0,0
42300132,7922679,8315278,8957631,0,9351666,c7155f38c0251cf4,42300133,0,423636,0,0,0,0,0,0,0,0
42300133,7932779,8315277,8405678,0,9351666,c7155f38c0251cf4,42300134,0,423636,0,0,0,0,0,0,0,0


In [29]:
save_cols = ['101','121','122','124','125','126','127','128','129'] + ['205','206','207','216','301'] + ['click']
train_all = train_all[save_cols]

In [30]:
train_all

,101,121,122,124,125,126,127,128,129,205,206,207,216,301,click
0,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887,4186222,8316799,8416205,9154780,9351665,0
1,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887,4186855,8316676,8482726,9188757,9351665,0
2,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887,4479003,8316590,8542197,9165876,9351665,1
3,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887,4933915,8316177,8529759,9323908,9351665,0
4,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887,4945663,8316589,8754197,9172179,9351665,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42300130,423636,0,0,0,0,0,0,0,0,7886537,8317499,9014187,0,9351666,0
42300131,423636,0,0,0,0,0,0,0,0,7918046,8315276,8532063,9318221,9351666,0
42300132,423636,0,0,0,0,0,0,0,0,7922679,8315278,8957631,0,9351666,0
42300133,423636,0,0,0,0,0,0,0,0,7932779,8315277,8405678,0,9351666,0


In [31]:
train_all.to_pickle(os.path.join(data_path, 'train_all.pkl'))

In [32]:
fea_test = pd.concat([pd.read_pickle(os.path.join(data_path, f'fea_test{i}.pkl')) for i in range(9)])
sample_test = pd.concat([pd.read_pickle(os.path.join(data_path, f'sample_test{i}.pkl')) for i in range(18)])

In [33]:
test_all = sample_test[item_features].merge(fea_test[user_features], how='left', on='common_feature_index')
test_all

,205,206,207,216,301,common_feature_index,sample_id,click,101,121,122,124,125,126,127,128,129
0,5587143,8315277,8801026,9181078,9351665,23bd0f75de327c60,1,0,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887
1,5662732,8316893,8987328,9206290,9351665,23bd0f75de327c60,2,0,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887
2,6539512,8315405,8546676,9273427,9351665,23bd0f75de327c60,3,0,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887
3,8010649,8315276,8801026,9133556,9351665,23bd0f75de327c60,4,0,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887
4,4018901,8317093,8863947,0,9351665,543b0cd53c7d5858,5,0,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43016835,7655278,8321614,8375868,0,9351667,64c9e607e7f490bb,43016836,0,197573,3438658,3438758,3438768,3438775,0,3438782,3864885,3864887
43016836,7660196,8317005,8707327,9293286,9351667,64c9e607e7f490bb,43016837,0,197573,3438658,3438758,3438768,3438775,0,3438782,3864885,3864887
43016837,7758388,8313679,8763077,9241291,9351667,64c9e607e7f490bb,43016838,0,197573,3438658,3438758,3438768,3438775,0,3438782,3864885,3864887
43016838,7893662,8313679,8915027,9230036,9351667,64c9e607e7f490bb,43016839,0,197573,3438658,3438758,3438768,3438775,0,3438782,3864885,3864887


In [34]:
save_cols = ['101','121','122','124','125','126','127','128','129'] + ['205','206','207','216','301'] + ['click']
test_all = test_all[save_cols]

In [35]:
test_all

,101,121,122,124,125,126,127,128,129,205,206,207,216,301,click
0,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887,5587143,8315277,8801026,9181078,9351665,0
1,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887,5662732,8316893,8987328,9206290,9351665,0
2,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887,6539512,8315405,8546676,9273427,9351665,0
3,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887,8010649,8315276,8801026,9133556,9351665,0
4,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887,4018901,8317093,8863947,0,9351665,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43016835,197573,3438658,3438758,3438768,3438775,0,3438782,3864885,3864887,7655278,8321614,8375868,0,9351667,0
43016836,197573,3438658,3438758,3438768,3438775,0,3438782,3864885,3864887,7660196,8317005,8707327,9293286,9351667,0
43016837,197573,3438658,3438758,3438768,3438775,0,3438782,3864885,3864887,7758388,8313679,8763077,9241291,9351667,0
43016838,197573,3438658,3438758,3438768,3438775,0,3438782,3864885,3864887,7893662,8313679,8915027,9230036,9351667,0


In [36]:
test_all.to_pickle(os.path.join(data_path, 'test_all.pkl'))

In [38]:
data = pd.concat([train_all, test_all]).reset_index(drop=True)
data

,101,121,122,124,125,126,127,128,129,205,206,207,216,301,click
0,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887,4186222,8316799,8416205,9154780,9351665,0
1,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887,4186855,8316676,8482726,9188757,9351665,0
2,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887,4479003,8316590,8542197,9165876,9351665,1
3,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887,4933915,8316177,8529759,9323908,9351665,0
4,31390,3438658,3438762,3438769,3438774,0,3438782,3864885,3864887,4945663,8316589,8754197,9172179,9351665,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85316970,197573,3438658,3438758,3438768,3438775,0,3438782,3864885,3864887,7655278,8321614,8375868,0,9351667,0
85316971,197573,3438658,3438758,3438768,3438775,0,3438782,3864885,3864887,7660196,8317005,8707327,9293286,9351667,0
85316972,197573,3438658,3438758,3438768,3438775,0,3438782,3864885,3864887,7758388,8313679,8763077,9241291,9351667,0
85316973,197573,3438658,3438758,3438768,3438775,0,3438782,3864885,3864887,7893662,8313679,8915027,9230036,9351667,0


In [39]:
data.isnull().sum()

101      0
121      0
122      0
124      0
125      0
126      0
127      0
128      0
129      0
205      0
206      0
207      0
216      0
301      0
click    0
dtype: int64

In [41]:
features = ['101','121','122','124','125','126','127','128','129'] + ['205','206','207','216','301']

In [42]:
from sklearn.preprocessing import OrdinalEncoder
data[features] = OrdinalEncoder().fit_transform(data[features])

In [45]:
data.describe()

,101,121,122,124,125,126,127,128,129,205,206,207,216,301,click
count,8.531698e+07,8.531698e+07,8.531698e+07,8.531698e+07,8.531698e+07,8.531698e+07,8.531698e+07,8.531698e+07,8.531698e+07,8.531698e+07,8.531698e+07,8.531698e+07,8.531698e+07,8.531698e+07,8.531698e+07
mean,2.211443e+05,1.624577e+01,6.286092e+00,1.499044e+00,3.854147e+00,7.361456e-01,2.426726e+00,9.149384e-01,1.611162e+00,2.425748e+06,3.619007e+03,3.456210e+05,7.734136e+04,6.296493e-01,3.888679e-02
std,1.290666e+05,2.599856e+01,3.536007e+00,7.105094e-01,1.842758e+00,9.492517e-01,1.052535e+00,4.031468e-01,1.418430e+00,1.211525e+06,1.702979e+03,2.035329e+05,7.604391e+04,4.981882e-01,1.933251e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.089600e+05,1.000000e+00,4.000000e+00,1.000000e+00,3.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,0.000000e+00,1.370386e+06,2.432000e+03,1.659430e+05,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.200880e+05,1.000000e+00,7.000000e+00,2.000000e+00,4.000000e+00,0.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,2.579759e+06,3.697000e+03,3.467470e+05,5.816800e+04,1.000000e+00,0.000000e+00
75%,3.325310e+05,2.700000e+01,8.000000e+00,2.000000e+00,5.000000e+00,2.000000e+00,3.000000e+00,1.000000e+00,3.000000e+00,3.493509e+06,4.032000e+03,5.229630e+05,1.414040e+05,1.000000e+00,0.000000e+00
max,4.448610e+05,9.700000e+01,1.300000e+01,2.000000e+00,7.000000e+00,3.000000e+00,3.000000e+00,2.000000e+00,4.000000e+00,4.348615e+06,8.993000e+03,6.951240e+05,2.348800e+05,2.000000e+00,1.000000e+00


In [43]:
data.to_pickle(os.path.join(data_path, 'data.pkl'))